# FinGPT-Forecaster Re-implemented with FinRobot

In this demo, we set up an agent to emulate the behavior of model in the fingpt-forecaster project with AutoGen, which takes a company's ticker symbol, recent basic financials and market news as input and predict its stock movements.

For detail of the original project, check out  [FinGPT-Forecaster](https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Forecaster)!  🔥[Demo](https://huggingface.co/spaces/FinGPT/FinGPT-Forecaster), [Medium Blog](https://medium.datadriveninvestor.com/introducing-fingpt-forecaster-the-future-of-robo-advisory-services-50add34e3d3c) & [Model](https://huggingface.co/FinGPT/fingpt-forecaster_dow30_llama2-7b_lora) on Huggingface🤗!

In [6]:
import autogen
from autogen.cache import Cache

from finrobot.utils import get_current_date, register_keys_from_json
from finrobot.data_source import FinnHubUtils, YFinanceUtils

After importing all the necessary packages and functions, we first instantiate a market analysis assistant and a user proxy using the agent classes defined by AutoGen. We also need the config for OpenAI & Finnhub here. 
- for openai configuration, rename OAI_CONFIG_LIST_sample to OAI_CONFIG_LIST and replace the api keys
- for finnhub configuration, rename config_api_keys_sample to config_api_keys and replace the api keys

In [7]:
# Read OpenAI API keys from a JSON file
config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST",
    # filter_dict={"model": ["gpt-4-0125-preview"]},
)
llm_config = {"config_list": config_list, "timeout": 120, "temperature": 0}

# Register FINNHUB API keys
register_keys_from_json("../config_api_keys")

In [8]:
analyst = autogen.AssistantAgent(
    name="Market_Analyst",
    system_message=" 作为一名市场分析师，必须具备较强的分析和解决问题的能力，收集所需的财务信息，并根据客户需求进行汇总。"
    "对于编程任务，只能使用你被提供的函数。任务完成后请回复 TERMINATE。",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get(
        "content", "").endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

Then we register our predefined functions with the userproxy.

Following our implementation with the fingpt-forecaster, we combined news/financials calls to the Finnhub API and stock data calls to the YFinance API.

In [75]:
from finrobot.toolkits import register_toolkits

tools = [
    {
        "function": FinnHubUtils.get_company_profile,
        "name": "get_company_profile",
        "description": "get a company's profile information"
    },
    {
        "function": FinnHubUtils.get_company_news,
        "name": "get_company_news",
        "description": "retrieve market news related to designated company"
    },
    {
        "function": FinnHubUtils.get_basic_financials,
        "name": "get_financial_basics",
        "description": "get latest financial basics for a designated company"
    },
    {
        "function": YFinanceUtils.get_stock_data,
        "name": "get_stock_data",
        "description": "retrieve stock price data for designated ticker symbol"
    }
]
register_toolkits(tools, 
                  caller = analyst, # The assistant agent can suggest calls to the calculator.
                  executor = user_proxy # The user proxy agent can execute the calculator calls.
                  )

Start the conversation, asking the agent to call all the functions, and see how it aggregates all the information and leads to the conclusion.

In [10]:
# company = "Tesla"
company = "APPLE"

with Cache.disk() as cache:
    # start the conversation
    user_proxy.initiate_chat(
        analyst,
        message=f"请使用所有可用工具，获取截至 {get_current_date()} 为止关于 {company} 的信息。请分析 {company} 的积极发展动态和潜在风险问题，"
        "分别列出 2-4 个最重要的因素，并保持简明扼要。多数因素应从公司相关新闻中推断得出。"
        f"然后对 {company} 的股价在下周的走势做出一个粗略预测（例如上涨/下跌 2-3%）。请提供一份总结分析来支持你的预测。",
        cache=cache,
    )

User_Proxy (to Market_Analyst):

请使用所有可用工具，获取截至 2025-06-02 为止关于 APPLE 的信息。请分析 APPLE 的积极发展动态和潜在风险问题，分别列出 2-4 个最重要的因素，并保持简明扼要。多数因素应从公司相关新闻中推断得出。然后对 APPLE 的股价在下周的走势做出一个粗略预测（例如上涨/下跌 2-3%）。请提供一份总结分析来支持你的预测。

--------------------------------------------------------------------------------
[autogen.oai.client: 06-02 11:49:15] {696} WARNING - Model qwen-turbo-2025-02-11 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Market_Analyst (to User_Proxy):


***** Suggested tool call (call_7ef0b59464d748efae7262): get_company_news *****
Arguments: 
{"symbol": "AAPL", "start_date": "2025-05-01", "end_date": "2025-06-02", "max_news_num": 10}
*******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_company_news...
Call ID: call_7ef0b59464d748efae7262
Input argument

## Data

### 1. `finnhub_utils`

Finnhub 提供的主要数据和服务包括：实时和历史市场数据、公司基本面数据、另类数据、技术分析数据、其他数据（全球 ETF、共同基金和指数数据、经济数据、美国债券数据）

#### `get_company_profile`

`formatted_str = (...) .format(**profile)`: 如果获取成功，它会将返回的 profile 字典中的数据填充到一个预设的格式化字符串模板中，生成一段易于阅读的公司介绍。

In [1]:
FinnHubUtils.get_company_profile("AAPL")

NameError: name 'FinnHubUtils' is not defined

In [43]:
finnhub_client = finnhub.Client(api_key=os.environ["FINNHUB_API_KEY"])
finnhub_client.company_profile2(symbol="AAPL")

{'country': 'US',
 'currency': 'USD',
 'estimateCurrency': 'USD',
 'exchange': 'NASDAQ NMS - GLOBAL MARKET',
 'finnhubIndustry': 'Technology',
 'ipo': '1980-12-12',
 'logo': 'https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAPL.png',
 'marketCapitalization': 2999860.6408355245,
 'name': 'Apple Inc',
 'phone': '14089961010',
 'shareOutstanding': 14935.83,
 'ticker': 'AAPL',
 'weburl': 'https://www.apple.com/'}

#### `get_company_news(...)`

检索与指定公司相关的市场新闻。（默认10条）

- 调用 `finnhub_client` 的 `company_news` 方法获取指定日期范围内的公司新闻。
  
- **对新闻数据进行处理：将时间戳 (`n["datetime"]`) 转换为 "YYYYMMDDHHMMSS" 格式的日期字符串，并提取标题 (`headline`) 和摘要 (`summary`)。**
  
- **如果获取到的新闻数量超过 max_news_num，则随机选择 max_news_num 条新闻。**
- **news.sort(key=lambda x: x["date"]): 按日期对新闻进行排序。**
- output = pd.DataFrame(news)



In [44]:
FinnHubUtils.get_company_news(
    symbol = "AAPL", 
    start_date = "2025-05-01", 
    end_date = "2025-06-02", 
    max_news_num = 10,
    save_path = "./AAPL_news.csv"
)

Finnhub client initialized
company news of AAPL saved to ./AAPL_news.csv


,date,headline,summary
0,20250524044100,"Stocks Fall on Tariff Broadside, After Week of...",President Trump renewed his tariff campaign Fr...
1,20250524045602,"Trump Targets Samsung, Apple Phones With 25% T...",(Bloomberg) -- Supply Lines is a daily newslet...
2,20250526081908,"Trump says US wants to make tanks, not T-shirts",U.S.President Donald Trump said on Sunday his ...
3,20250527000000,"How To Build A $100,000 Dividend Portfolio Tha...",The Dividend Income Accelerator Portfolio targ...
4,20250527182700,"As Trump hits E.U., phone makers with fresh th...","As Trump hits E.U., phone makers with fresh th..."
5,20250528041500,EU Crisis Averted. Same Time Next Week?,Wall Street sprinted back from last weekâs s...
6,20250528045846,iPhone Tariff Threat Weighs on Apple,"It's a ""fairytale"" to think the full iPhone su..."
7,20250528233406,Apple to Debut New Gaming App,Apple plans to launch a new gaming app at its ...
8,20250529043030,Trump's Apple Beef Isn't Just About India-Made...,Apple Inc. (NASDAQ:AAPL) CEO Tim Cook has foun...
9,20250530120305,Texas passes law requiring age verification in...,Copyright © BusinessAMBE 2023Key takeawaysTexa...


In [49]:
news = finnhub_client.company_news("AAPL", _from="2025-05-01", to="2025-06-02")

print(len(news))

news

235


[{'category': 'company',
  'datetime': 1748824690,
  'headline': 'Apple To Appeal EU Requirement To Share Info With Tech Rivals- Axios',
  'id': 134930923,
  'image': '',
  'related': 'AAPL',
  'source': 'Finnhub',
  'summary': 'June 2 - * APPLE TO APPEAL EU REQUIREMENT TO SHARE INFO WITH TECHRIVALS-AXIOSSource text: https://tinyurl.com/yk8cndedFurther company coverage: ...',
  'url': 'https://finnhub.io/api/news?id=19519f0fe4a1f4f328a62c14512d61f82a3a951a6895f4e2923cb61024814bee'},
 {'category': 'company',
  'datetime': 1748822803,
  'headline': 'Stock market today: Dow, S&P 500, Nasdaq futures slip as Wall Street kicks off June after bullish May',
  'id': 134929798,
  'image': 'https://s.yimg.com/rz/stage/p/yahoo_finance_en-US_h_p_finance_2.png',
  'related': 'AAPL',
  'source': 'Yahoo',
  'summary': 'US stock futures edged lower Sunday evening, following a bullish May driven by tech optimism and resilient economic data.',
  'url': 'https://finnhub.io/api/news?id=fe9602b152d802e98223

#### `get_basic_financials_history(...)`

- `finnhub_client.company_basic_financials(symbol, "all")`: 获取指定公司的所有基本财务数据.
  
- 使用 `defaultdict(dict)` 来组织数据。遍历从API获取的特定频率 (`freq`) 的财务指标 (`metric`) 和对应的值列表 (`value_list`)。
- 返回dataframe


In [54]:
FinnHubUtils.get_basic_financials_history(
    symbol="AAPL",
    freq="quarterly",
    start_date="2022-06-01",
    end_date="2025-06-01",
    selected_columns=[
        "eps", "grossMargin", "netMargin", "roaTTM", "roeTTM", "peTTM"
    ],
    save_path="./AAPL_financials_quarterly.csv"
)

Finnhub client initialized
basic financials saved to ./AAPL_financials_quarterly.csv


,eps,grossMargin,netMargin,peTTM,roaTTM,roeTTM
date,,,,,,
2025-03-29,1.6458,0.4705,0.2599,34.3327,0.2837,1.5131
2024-12-28,2.3979,0.4688,0.2923,39.6486,0.2791,1.4535
2024-09-28,0.9667,0.4622,0.1552,37.5759,0.2702,1.3787
2024-06-29,1.3974,0.4626,0.2500,32.3613,0.2966,1.4715
2024-03-30,1.5284,0.4658,0.2604,26.3771,0.2913,1.4833
2023-12-30,2.1774,0.4587,0.2836,29.6728,0.2939,1.5604
2023-09-30,1.4647,0.4517,0.2565,27.8553,0.2839,1.6078
2023-07-01,1.2603,0.4452,0.2431,31.7812,0.2773,1.6492
2023-04-01,1.5246,0.4426,0.2548,27.7007,0.2758,1.6572


In [65]:
basic_financials = finnhub_client.company_basic_financials("AAPL", "all")

import json
print(json.dumps(basic_financials, indent=2))

{
  "metric": {
    "10DayAverageTradingVolume": 55.16882,
    "13WeekPriceReturnDaily": -15.3603,
    "26WeekPriceReturnDaily": -12.2926,
    "3MonthADReturnStd": 51.084312,
    "3MonthAverageTradingVolume": 59.25201,
    "52WeekHigh": 260.1,
    "52WeekHighDate": "2024-12-26",
    "52WeekLow": 169.2101,
    "52WeekLowDate": "2025-04-08",
    "52WeekPriceReturnDaily": 5.5494,
    "5DayPriceReturnDaily": 2.8576,
    "assetTurnoverAnnual": 1.0714,
    "assetTurnoverTTM": 1.1673,
    "beta": 1.2146548,
    "bookValuePerShareAnnual": 3.7673,
    "bookValuePerShareQuarterly": 4.4712,
    "bookValueShareGrowth5Y": -5.85,
    "capexCagr5Y": -2.08,
    "cashFlowPerShareAnnual": 7.1978,
    "cashFlowPerShareQuarterly": 6.5924,
    "cashFlowPerShareTTM": 6.86253,
    "cashPerSharePerShareAnnual": 4.3112,
    "cashPerSharePerShareQuarterly": 3.2463,
    "currentDividendYieldTTM": 0.5105,
    "currentEv/freeCashFlowAnnual": 28.214,
    "currentEv/freeCashFlowTTM": 31.1708,
    "currentRatioAnnual

#### `get_basic_financials(...)`

- `basic_financials = finnhub_client.company_basic_financials(symbol, "all")`: 获取所有基本财务数据

- `output_dict = basic_financials["metric"]`: 初始化 `output_dict`，使用从API返回的 `metric` 字段，这通常包含一些最新的、非时间序列的指标值。

- 遍历 `basic_financials["series"]["quarterly"].items()`: 获取最新的季度数据（通常是列表中的第一个元素 value_list[0]），并用这些值更新 output_dict。

- `return json.dumps(output_dict, indent=2)`: 将最终的字典转换为格式化的 JSON 字符串


In [73]:
print(FinnHubUtils.get_basic_financials("AAPL"))

Finnhub client initialized
{
  "10DayAverageTradingVolume": 55.16882,
  "13WeekPriceReturnDaily": -15.3603,
  "26WeekPriceReturnDaily": -12.2926,
  "3MonthADReturnStd": 51.084312,
  "3MonthAverageTradingVolume": 59.25201,
  "52WeekHigh": 260.1,
  "52WeekHighDate": "2024-12-26",
  "52WeekLow": 169.2101,
  "52WeekLowDate": "2025-04-08",
  "52WeekPriceReturnDaily": 5.5494,
  "5DayPriceReturnDaily": 2.8576,
  "assetTurnoverAnnual": 1.0714,
  "assetTurnoverTTM": 1.1673,
  "beta": 1.2146548,
  "bookValuePerShareAnnual": 3.7673,
  "bookValuePerShareQuarterly": 4.4712,
  "bookValueShareGrowth5Y": -5.85,
  "capexCagr5Y": -2.08,
  "cashFlowPerShareAnnual": 7.1978,
  "cashFlowPerShareQuarterly": 6.5924,
  "cashFlowPerShareTTM": 6.86253,
  "cashPerSharePerShareAnnual": 4.3112,
  "cashPerSharePerShareQuarterly": 3.2463,
  "currentDividendYieldTTM": 0.5105,
  "currentEv/freeCashFlowAnnual": 28.214,
  "currentEv/freeCashFlowTTM": 31.1708,
  "currentRatioAnnual": 0.8673,
  "currentRatioQuarterly": 0.8

In [68]:
basic_financials = finnhub_client.company_basic_financials("AAPL", "all")

import json
print(json.dumps(basic_financials, indent=2))

{
  "metric": {
    "10DayAverageTradingVolume": 55.16882,
    "13WeekPriceReturnDaily": -15.3603,
    "26WeekPriceReturnDaily": -12.2926,
    "3MonthADReturnStd": 51.084312,
    "3MonthAverageTradingVolume": 59.25201,
    "52WeekHigh": 260.1,
    "52WeekHighDate": "2024-12-26",
    "52WeekLow": 169.2101,
    "52WeekLowDate": "2025-04-08",
    "52WeekPriceReturnDaily": 5.5494,
    "5DayPriceReturnDaily": 2.8576,
    "assetTurnoverAnnual": 1.0714,
    "assetTurnoverTTM": 1.1673,
    "beta": 1.2146548,
    "bookValuePerShareAnnual": 3.7673,
    "bookValuePerShareQuarterly": 4.4712,
    "bookValueShareGrowth5Y": -5.85,
    "capexCagr5Y": -2.08,
    "cashFlowPerShareAnnual": 7.1978,
    "cashFlowPerShareQuarterly": 6.5924,
    "cashFlowPerShareTTM": 6.86253,
    "cashPerSharePerShareAnnual": 4.3112,
    "cashPerSharePerShareQuarterly": 3.2463,
    "currentDividendYieldTTM": 0.5105,
    "currentEv/freeCashFlowAnnual": 28.214,
    "currentEv/freeCashFlowTTM": 31.1708,
    "currentRatioAnnual

### 2. `yfinance_utils`

这里面给Agent注册的工具只有`get_stock_data(...)`

- `get_stock_data(...)`: 获取指定股票代码 (`symbol`) 在给定开始日期 (`start_date`) 和结束日期 (`end_date`) 之间的历史股价数据。返回一个包含股价数据的 `DataFrame`.

- `get_stock_info(...)`: 获取股票的最新信息。返回包含股票信息的字典。

- `get_company_info(...)`: 获取并返回公司的基本信息（如公司名、行业、国家等）到一个 `DataFrame`

- `get_stock_dividends(...)`: 获取并返回最新的股息数据。返回一个包含股息数据的 `DataFrame`。

- `get_income_stmt(...)`: 获取并返回公司最新的利润表. 返回一个包含利润表数据的 `DataFrame`

- `get_balance_sheet(...)`: 获取并返回公司最新的资产负债表. 返回一个包含资产负债表数据的 `DataFrame`

- `get_cash_flow(...)`: 获取并返回公司最新的现金流量表. 返回一个包含现金流量表数据的 `DataFrame`

- `get_analyst_recommendations(...)`: 获取最新的分析师评级建议。返回一个tuple，包含最常见的建议（字符串）和它的票数（整数）。如果无建议，返回 (None, 0)。

#### `get_stock_data`


In [76]:
YFinanceUtils.get_stock_data(
    symbol = "AAPL", 
    start_date = "2025-05-01", 
    end_date = "2025-06-02", 
    save_path = "./AAPL_news.csv"
)

Stock data for AAPL saved to ./AAPL_news.csv


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-05-01 00:00:00-04:00,208.806182,214.279000,208.626409,213.040634,57365700,0.00,0.0
2025-05-02 00:00:00-04:00,205.820091,206.718922,201.895245,205.081070,101010600,0.00,0.0
2025-05-05 00:00:00-04:00,202.834025,203.832716,197.950430,198.629532,69018500,0.00,0.0
2025-05-06 00:00:00-04:00,197.950420,200.387212,196.761976,198.250015,51216500,0.00,0.0
2025-05-07 00:00:00-04:00,198.909155,199.178806,192.996910,195.992981,68536700,0.00,0.0
2025-05-08 00:00:00-04:00,197.461064,199.788014,194.425036,197.231369,50478900,0.00,0.0
2025-05-09 00:00:00-04:00,198.739390,200.277366,197.281295,198.270004,36453900,0.00,0.0
2025-05-12 00:00:00-04:00,210.970001,211.270004,206.750000,210.789993,63775800,0.26,0.0
2025-05-13 00:00:00-04:00,210.429993,213.399994,209.000000,212.929993,51909300,0.00,0.0


In [77]:
YFinanceUtils.get_analyst_recommendations("AAPL")

('buy', 20)